In [1]:
! pip install -q --upgrade google-generativeai langchain-google-genai chromadb pypdf

In [2]:
! pip install langchain

In [1]:
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import Chroma
from langchain.chains.question_answering import load_qa_chain
# from langchain.chains import RetrievalQA
from langchain import PromptTemplate

from langchain_google_genai import ChatGoogleGenerativeAI

In [2]:
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)

In [3]:
Gemini_pro_llm = ChatGoogleGenerativeAI(model="gemini-pro",
                                        google_api_key="<API-KEY>",
                                        temperature=1.0,
                                        convert_system_message_to_human=True,
                                        safety_settings={
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
    })

In [4]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [5]:
GeminiEmbeddingModel = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key="<API-KEY>")

In [ ]:
# For PDF FILES
pdfLoader = PyPDFLoader("/content/2020- July-M.Tech.( CSE)-Regulations_Final.pdf")
pdf_pages = pdfLoader.load_and_split()

In [ ]:
print(pdf_pages[1].page_content)

2 
 and Research Methods (2 credits).  Requirements of 48 credits for all the three  options are as 
follows ;  
 
 
a. M.Tech.  with thesis ; 32 credits of course work + 16 credits of thesis. At most 4 credits 
may be earned by doing 300 and 400 level courses.  
b. M.Tech.   with Scholarly Paper ; 40 or 44 credits of course work + 8  or 4 credits for a 
scholarly paper . At most 8 credits may be earned by doing 300 and 400 level courses.  
c. M.Tech . without thesis and scholarly paper option ; 48 credits of course work.  
 
 
3. For the thesis or the scholarly paper credits, the student has to register  but need not be physically 
present and can do the work while being outside the Institute.  
 
4. A student admitted to the M.Tech.  program will give his/her choice regarding whether he/she wants 
to pursue  the thesis or scholarly paper or without thesis /scholarly paper  option . However, this 
choice can be changed at any time during the program by suitably informing the PG Commit

In [ ]:
len(pdf_pages)

4

In [ ]:
textSplitter = RecursiveCharacterTextSplitter(chunk_size=10000,chunk_overlap = 1000)


In [ ]:
context = "\n\n".join(str(page.page_content) for page in pdf_pages)
text = textSplitter.split_text(context)

In [ ]:
chroma_PDF_VectorIndex = Chroma.from_texts(text, GeminiEmbeddingModel).as_retriever(search_kwargs={"k":5})

In [ ]:
QA_PDF_chain = RetrievalQA.from_chain_type(
    Gemini_pro_llm,
    retriever=chroma_PDF_VectorIndex,
    return_source_documents=True

)

In [8]:
! pip install beautifulsoup4

In [6]:
from langchain_community.document_loaders import WebBaseLoader
web_loader = WebBaseLoader(["https://en.wikipedia.org/wiki/Sport_in_India",
                            "https://www.indianetzone.com/28/memorable_events_indian_cricket.htm",])
documents = web_loader.load()

In [7]:
# Trying Recursive loader
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader

In [ ]:
from bs4 import BeautifulSoup as Soup

url = "https://en.wikipedia.org/wiki/Sport_in_India"
recurrsive_loader = RecursiveUrlLoader(
    url=url, max_depth=2, extractor=lambda x: Soup(x, "html.parser").text
)
documents = recurrsive_loader.load()

In [7]:
text_splitter = RecursiveCharacterTextSplitter()
splitted_documents = text_splitter.split_documents(documents)
vector_index = Chroma.from_documents(splitted_documents, GeminiEmbeddingModel)

In [8]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain


prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_retrieval_chain = create_stuff_documents_chain(Gemini_pro_llm, prompt)

In [9]:
from langchain.chains import create_retrieval_chain

retriever = vector_index.as_retriever()
Question_Answer_Chain = create_retrieval_chain(retriever, document_retrieval_chain)

In [10]:
response = Question_Answer_Chain.invoke({"input": "who is the sachin tendulkar ?"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...

Widely considered as one of the greatest batter of all time


In [11]:
import gradio as gr
# "What are the Requirements for M.tech Specialization. ?"
def ask_Query(Query):
    question = Query
    result = Question_Answer_Chain.invoke({"input": question})
    return result["answer"]

In [12]:
!pip install gradio

In [12]:
UI_Interface = gr.Interface(
    fn=ask_Query,
    inputs=gr.Textbox(lines=2, placeholder="Ask Query Here"),
    outputs="text",
)


In [13]:
UI_Interface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://b2ffa90129e84ea947.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
